In [8]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [9]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [10]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()


In [11]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=390)  

# Twelve Data API parameters
symbol = "SPX"
interval = "1min"
outputsize = 5000  
parent_df = pd.DataFrame()  # Initialize an empty parent DataFrame

# Increment through the time range in start_date and end_date by 5 days every 30 seconds
while start_date < end_date:
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=(start_date + timedelta(days=5)).strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)
    
    # Concatenate the current time series DataFrame to the parent DataFrame
    parent_df = pd.concat([parent_df, time_series_df], ignore_index=True)
 
    # Increment the start_date
    start_date += timedelta(days=5)
    
    # Sleep for 30 seconds
    time.sleep(30)

# Print the parent DataFrame to a CSV file
parent_df.to_csv('aux_historical_data.csv', index=False)
